# 0. 필요한 패키지 불러오기

In [25]:
import nltk
import librosa
import torch
import soundfile as sf
from transformers import Wav2Vec2ForCTC, Wav2Vec2Tokenizer
# nltk.download('punkt')

# 1. 단계별 함수 정의

In [26]:
# 모델 로드

def stt_load_model():
    # model, tokenizer 로드 
    tokenizer = Wav2Vec2Tokenizer.from_pretrained("facebook/wav2vec2-base-960h")
    model = Wav2Vec2ForCTC.from_pretrained("facebook/wav2vec2-base-960h")    
    return tokenizer, model

In [27]:
# 데이터 읽고 전처리

def stt_transcript(tokenizer, model, speech, samplerate):
    speech, samplerate = sf.read(input_file)
    # multi-channel -> mono
    if len(speech.shape) > 1: 
#         speech = speech[:,0] + speech[:,1]
        speech = speech.mean(axis=1)
    # 16khz로 Resampling
    if samplerate != 16000:
        speech = librosa.resample(speech, samplerate, 16000)
    # 토크나이징
    input_values = tokenizer(speech, return_tensors="pt").input_values
    # logits
    logits = model(input_values).logits
    # argmax -> 가장 높은 확률의 단어 반환
    predicted_ids = torch.argmax(logits, dim=-1)
    # 디코딩
    transcription = tokenizer.decode(predicted_ids[0])
    #  punkt 모델을 사용해 sentence tokenization (문장의 앞 글자만 대문자)
    sentences = nltk.sent_tokenize(transcription.lower())
    transcription = (' '.join([s.replace(s[0],s[0].capitalize(),1) for s in sentences]))
    
    return transcription

# 2. 사용자가 데이터 업로드 시

In [28]:
wav_input = 'harvard.wav'

# 3. 모델, 토크나이저 로드

In [29]:
tokenizer, model = stt_load_model()

/opt/conda/lib/python3.8/site-packages/transformers/models/wav2vec2/tokenization_wav2vec2.py:417: FutureWarning: The class `Wav2Vec2Tokenizer` is deprecated and will be removed in version 5 of Transformers. Please use `Wav2Vec2Processor` or `Wav2Vec2CTCTokenizer` instead.
  warnings.warn(
Some weights of Wav2Vec2ForCTC were not initialized from the model checkpoint at facebook/wav2vec2-base-960h and are newly initialized: ['wav2vec2.masked_spec_embed']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# 4. 전처리

In [30]:
text = stt_transcript(tokenizer,model,wav_input)

# 5. 결과 출력

In [31]:
print(text)

The stale smell of old beer lingers it takes heat to bring out the odor a cold dip restores health and zest a salt pickle tastes fine with ham takos al pastore are my favorite a zestful food is the hot cross bun
